# Construct Perpendicular Vector

This notebook constructs a vector orthogonal to V_complexity for 2D activation space exploration. We know in advance that the V_complexity we want is the mean difference vector from the model's last layer.

**Method:** Gram-Schmidt orthogonalization in float32 precision with random seed 42.

**Output:** `vectors/perpendicular_vector.pt` - V_perpendicular as bfloat16 tensor

In [1]:
import torch
import numpy as np

## Load the complexity vector

In [2]:
# Load all vectors
vectors = torch.load('vectors/complexity_vectors.pt')

# Extract layer 27 (highest magnitude)
v_complexity = vectors[27]

print(f"Shape: {v_complexity.shape}")
print(f"Dtype: {v_complexity.dtype}")
print(f"L2 norm: {torch.norm(v_complexity).item():.4f}")

Shape: torch.Size([3072])
Dtype: torch.bfloat16
L2 norm: 9.8750


## Construct an orthogonal vector

We'll use Gram-Schmidt orthogonalization in float32 precision to construct a vector orthogonal to V_complexity.

In [3]:
# Convert to float32
v_complexity_f32 = v_complexity.float()
v_random_f32 = torch.randn_like(v_complexity_f32)

# Set seed for reproducibility
torch.manual_seed(42)
v_random_f32 = torch.randn_like(v_complexity_f32)

# Gram-Schmidt in float32
dot_product_f32 = torch.dot(v_random_f32, v_complexity_f32)
norm_squared_f32 = torch.dot(v_complexity_f32, v_complexity_f32)
projection_f32 = (dot_product_f32 / norm_squared_f32) * v_complexity_f32
v_perpendicular_f32 = v_random_f32 - projection_f32

# Check orthogonality
orthogonality_check_f32 = torch.dot(v_complexity_f32, v_perpendicular_f32).item()

print(f"V_perpendicular (f32) L2 norm: {torch.norm(v_perpendicular_f32).item():.4f}")
print(f"Dot product (should be ~0): {orthogonality_check_f32}")
print(f"Orthogonal? {abs(orthogonality_check_f32) < 1e-4}")

V_perpendicular (f32) L2 norm: 55.0698
Dot product (should be ~0): 9.5367431640625e-07
Orthogonal? True


## Save the perpendicular vector

Convert back to bfloat16 to match V_complexity and save for downstream use.

In [4]:
# Convert back to bfloat16 to match the complexity vector
v_perpendicular_bf16 = v_perpendicular_f32.bfloat16()

# Save to file
torch.save(v_perpendicular_bf16, 'vectors/perpendicular_vector.pt')

print(f"Saved V_perpendicular to vectors/perpendicular_vector.pt")
print(f"Shape: {v_perpendicular_bf16.shape}")
print(f"Dtype: {v_perpendicular_bf16.dtype}")
print(f"L2 norm: {torch.norm(v_perpendicular_bf16).item():.4f}")

Saved V_perpendicular to vectors/perpendicular_vector.pt
Shape: torch.Size([3072])
Dtype: torch.bfloat16
L2 norm: 55.0000
